In [198]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [124]:
data_folder = 'data'
if not os.path.exists(data_folder):
    os.mkdir(data_folder)

In [125]:
branch_mapping = """
	<option value="15">An Giang</option>
	<option selected="selected" value="61">Ba Đình</option>
	<option value="84">Bắc Bình Dương</option>
	<option value="73">Bắc Giang</option>
	<option value="105">Bắc Gia Lai</option>
	<option value="64">Bắc Hà Tĩnh</option>
	<option value="89">Bạc Liêu</option>
	<option value="35">Bắc Ninh</option>
	<option value="50">Bắc Sài Gòn</option>
	<option value="24">Bến Tre</option>
	<option value="48">Biên Hòa</option>
	<option value="5">Bình Định</option>
	<option value="28">Bình Dương</option>
	<option value="104">Bình Phước</option>
	<option value="25">Bình Tây</option>
	<option value="62">Bình Thuận</option>
	<option value="19">Cà Mau</option>
	<option value="11">Cần Thơ</option>
	<option value="47">Châu Đốc</option>
	<option value="54">Chương Dương</option>
	<option value="4">Đà Nẵng</option>
	<option value="23">Đắc Lắc</option>
	<option value="96">Đông Anh</option>
	<option value="107">Đông Bình Dương</option>
	<option value="12">Đồng Nai</option>
	<option value="53">Đông Sài Gòn</option>
	<option value="60">Đồng Tháp</option>
	<option value="57">Dung Quất</option>
	<option value="88">Gia Định</option>
	<option value="29">Gia Lai</option>
	<option value="91">Tân Sơn Nhất</option>
	<option value="52">Hạ Long</option>
	<option value="90">Hà Nam</option>
	<option value="2">Hà Nội</option>
	<option value="69">Tây Hà Nội</option>
	<option value="85">Hà Thành</option>
	<option value="20">Hà Tĩnh</option>
	<option value="34">Hải Dương</option>
	<option value="3">Hải Phòng</option>
	<option value="30">Hoàn Kiếm</option>
	<option value="93">Hoàng Mai</option>
	<option value="68">Hội Sở Chính</option>
	<option value="16">Huế</option>
	<option value="42">Hùng Vương</option>
	<option value="59">Hưng Yên</option>
	<option value="6">Khánh Hòa</option>
	<option value="9">Kiên Giang</option>
	<option value="76">Kon Tum</option>
	<option value="72">Kỳ Đồng</option>
	<option value="56">Lâm Đồng</option>
	<option value="98">Lạng Sơn</option>
	<option value="95">Lào Cai</option>
	<option value="63">Long An</option>
	<option value="87">Long Khánh</option>
	<option value="55">Móng Cái</option>
	<option value="41">Nam Bình Dương</option>
	<option value="100">Nam Đà Nẵng</option>
	<option value="83">Nam Định</option>
	<option value="97">Nam Hà Nội</option>
	<option value="103">Nam Hải Phòng</option>
	<option value="18">Nam Sài Gòn</option>
	<option value="10">Nghệ An</option>
	<option value="111">Nghi Sơn</option>
	<option value="92">Phú Nhuận</option>
	<option value="58">Nha Trang</option>
	<option value="40">Nhơn Trạch</option>
	<option value="22">Ninh Bình</option>
	<option value="81">Ninh Thuận</option>
	<option value="102">Phố Hiến</option>
	<option value="101">Phú Quốc</option>
	<option value="80">Phú Thọ</option>
	<option value="75">Phú Yên</option>
	<option value="109">Phúc Yên</option>
	<option value="31">Quảng Bình</option>
	<option value="65">Quảng Nam</option>
	<option value="27">Quảng Ngãi</option>
	<option value="14">Quảng Ninh</option>
	<option value="77">Quảng Trị</option>
	<option value="43">Quy Nhơn</option>
	<option value="33">Sài Gòn</option>
	<option value="51">Sài Thành</option>
	<option value="1">Sở Giao Dịch</option>
	<option value="94">Sóc Sơn</option>
	<option value="32">Sóc Trăng</option>
	<option value="46">Tân Bình Dương</option>
	<option value="44">Tân Bình</option>
	<option value="37">Tân Định</option>
	<option value="39">Tây Cần Thơ</option>
	<option value="99">Tây Hồ</option>
	<option value="70">Tây Ninh</option>
	<option value="17">Tây Sài Gòn</option>
	<option value="21">Thái Bình</option>
	<option value="82">Thái Nguyên</option>
	<option value="49">Thăng Long</option>
	<option value="45">Thành Công</option>
	<option value="78">Thanh Hóa</option>
	<option value="71">Thanh Xuân</option>
	<option value="38">Thủ Đức</option>
	<option value="26">Thủ Thiêm</option>
	<option value="67">Tiền Giang</option>
	<option value="7">TP HCM</option>
	<option value="74">Trà Vinh</option>
	<option value="86">Vinh</option>
	<option value="79">Vĩnh Long</option>
	<option value="36">Vĩnh Phúc</option>
	<option value="8">Vũng Tàu</option>
	<option value="112">Tuyên Quang</option>
"""

In [126]:
arr = []
for x in filter(lambda x: x, branch_mapping.split('\n')):
    x = x.strip()
    value = int(re.findall('value="([\w]*)"?', x)[0])
    city = re.findall(".*>(.*?)<", x)[0]
    arr.append({'value': value, 'city': city})
city_mapping = pd.DataFrame(arr)
city_mapping.to_csv('{}/city_mapping.csv'.format(data_folder), index=False, encoding='utf-8')

In [127]:
def request_content(branchID, date):
    cookies = {
        'ASP.NET_SessionId': 'n5ijcnsym0nk3cz5ftjoxij5',
    }

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Origin': 'https://www.vietcombank.com.vn',
        'Upgrade-Insecure-Requests': '1',
        'Content-Type': 'application/x-www-form-urlencoded',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Referer': 'https://www.vietcombank.com.vn/ExchangeRates/?lang=en',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    params = (
        ('lang', 'en'),
    )

    data = {
      '__VIEWSTATE': 'GCtcDuh2SE292rZ/JtcBejgJ0FE5WFlk3fDCmTofjbpeRw36+qxt4N8ZYR0edRaiWYe7uY9OVaYWZyW8tBvs6361s9Nr7vpH+vtE9GkOalwZ4UP1H6coek2PXIzcVbn/mZr8f3OEYt3Bt+JmVhCU08gMnHW9SlHCyCOQwWY97gqYflYfyQIZBxv+CzO9K0ZqIRrQejyTTrANj29Et6bnCKonzUoLgOXVaDqOpXEh4ZkjOqUnV/LjURRjwYEndfwVXvB5PNdStP92l8xm7CTdaixvlWO+QwU+Mk825DOib1YZkMJba6e6fi1D8AJMRBU04pY1NZcG2B2vFXy5D1+0FFvyYqgxfrbUWsreCf4L2xKiUnJwWladHTdaEqS6gwTSOOySZVlR50pr1JCIGDsMLN8rVQum6hWVqNYEI+opQYFHGeHW89Z9UadA4s2H5TaMSbYDWw60eOQb9TN2YECfTVrGvZxYdmoGFq9m5vmx/5krTqgRi7ZFqoQOsd/49QlYAFgokKtERfNPdTtw6LNDSA2g0sHTlTkMUJwmltbIEqWK8ay1UDDZExG7Yrw2RQShyBSbTop2DHXiMJRB/UOXmro2NvL/6/LBZy+HNJAQ8Ocsw3hug3/XLumLjtE0KllL8e7hZeimatgj07hhq85ThfNRhRz7bSzBblrvNYjztsS2jOzNUb86D9rLhtJ0DesuMlQgcHaC6U53nDvt5kD3FvwSgCR6mn6YyyRGAYMCjkZYsdD2NbNWCXWK8TMiOUyPzlVu4xUUrzvzj22upuqIh2A/1TjTHdnN1usYWjOxcfkStYPgKPSq2oR0UXxhhPuBswBiLEkHZj5U+292ukv7dCnSk6P4Kti0OZ4C5ib/p2y+bo9hKTVsw6QkCTv6fhnkOAzPMjVfrorsiFn1gLcuDC43RQVOoYlffSvC3xQFw/U4Fit8LoaQUaI+ds7ciOYVp0FInIWEGYqH/hEIzgigg0f4cuHY66KshQZdqlM8mgh7CfRlXMgmPfyI+lIOszlFZNlx/SuNjo45XtToKG8EG6w6tl+OzU1O+426EnG01w1H+yEbSNDftpjvEvxXiB8QQbLG7WNjZRNo8PjNmGkw3zqFaxpZDgKcWJZ/RoDMn1LhNZJ7S2MRmE0onknE9RtKOaTP/nXcH6c0eYdw/YshqY+f146CAafN20zxYr7A8Vlowu+FCqdZ0aw/nDMI/OGcVnDjmIffslM3BNyKSxEM2FEy/SxImYs6lg2Sr8Z54nFMWk8ctkUrGsNEfICBhyS4RD86TJsoYqVcouLKz7f8DjMWvRA/H7vxq9E9DuKuOm3DQANc575EQ9lOXYcY1QsvwhfNdQ==',
      '__VIEWSTATEGENERATOR': '6B3FDDEB',
      '__VIEWSTATEENCRYPTED': '',
      'ctl00$Content$BranchList': branchID,
      'ctl00$Content$DateText': date,
      'ctl00$Content$ViewButton': 'View'
    }

    response = requests.post('https://www.vietcombank.com.vn/ExchangeRates/', headers=headers, params=params, cookies=cookies, data=data)
    return response.content

In [197]:
def parse_data(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    table_data = soup.find("table", {"id": "Content_ExrateView"})
    headers = table_data.find_all('th')
    headers = [x.text for x in headers]
    arr = []
    for rows in table_data.find_all('tr'):
        cols = [x.text.strip() for x in rows.find_all('td')]
        cols = [x if x != '-' else None for x in cols]
        if len(cols) > 0:
            d = dict(zip(headers, cols))
            arr.append(d)
    df = pd.DataFrame(arr)
    return df



df.melt(id_vars='Currency Code', value_vars=['Buy', 'Sell', 'Transfer'])

,Currency Code,variable,value
0,AUD,Buy,"16,247.81"
1,CAD,Buy,"17,343.19"
2,CHF,Buy,"22,929.61"
3,DKK,Buy,None
4,EUR,Buy,"26,162.38"
5,GBP,Buy,"30,382.96"
6,HKD,Buy,"2,909.14"
7,INR,Buy,None
8,JPY,Buy,196.21
9,KRW,Buy,18.96


In [220]:
dates = pd.date_range('2019-01-01', '2019-03-01')#, datetime.now())
for date in dates:
    arr_df = []
    query_date = date.strftime('%d/%m/%Y')
    date = str(date.date())
    for branchID in city_mapping['value'].values:
        html_doc = request_content(str(branchID), query_date)
        df = parse_data(html_doc)
        df['date'] = date
        df['branchID'] = branchID
        arr_df.append(df)
    df = pd.concat(arr_df)
    filename = '{}/{}.csv'.format(data_folder, date)
    df.to_csv(filename)
    print('save', df.shape[0], 'rows to ', filename)

save 1876 rows to  data/2019-01-01.csv
save 1876 rows to  data/2019-01-02.csv
save 1876 rows to  data/2019-01-03.csv
save 1876 rows to  data/2019-01-04.csv
save 1876 rows to  data/2019-01-05.csv
save 1876 rows to  data/2019-01-06.csv
save 1876 rows to  data/2019-01-07.csv
save 1876 rows to  data/2019-01-08.csv
save 1876 rows to  data/2019-01-09.csv
save 1876 rows to  data/2019-01-10.csv
save 1876 rows to  data/2019-01-11.csv
save 1876 rows to  data/2019-01-12.csv
save 1876 rows to  data/2019-01-13.csv
save 1876 rows to  data/2019-01-14.csv
save 1876 rows to  data/2019-01-15.csv
save 1876 rows to  data/2019-01-16.csv
save 1877 rows to  data/2019-01-17.csv
save 1876 rows to  data/2019-01-18.csv
save 1876 rows to  data/2019-01-19.csv
save 1876 rows to  data/2019-01-20.csv
save 1876 rows to  data/2019-01-21.csv
save 1876 rows to  data/2019-01-22.csv
save 1876 rows to  data/2019-01-23.csv
save 1876 rows to  data/2019-01-24.csv
save 1876 rows to  data/2019-01-25.csv
save 1876 rows to  data/2